In [ ]:
!pip install focal-loss

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import pylab
import cv2
import os

from focal_loss import BinaryFocalLoss

import warnings
warnings.filterwarnings("ignore")

In [3]:
TestData = pd.DataFrame(columns=['DIR', 'Fever/MP', 'ORC'])
TestData['DIR'] = ['/content/Track 5.m4a'] #Path of your cough file
TestData['Fever/MP'] = [0] #Do you have a fever or muscle pain? Yes: 1, No: 0
TestData['ORC'] = [0] #Do you have respiratory diseases? Yes: 1, No: 0
TestData

,DIR,Fever/MP,ORC
0,/content/Track 5.m4a,0,0


---
# Prepare Data
---

In [4]:
custpath1  = '/content/custom_dataset_test/'

def feature_extractor(row):

  name     = row[0]
  try:
    audio,sr = librosa.load(row[0])
    #For MFCCS 
    mfccs    = librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=13)
    mfccsscaled = np.mean(mfccs.T,axis=0)
    
    #Mel Spectogram
    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
    melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
    s_db     = librosa.power_to_db(melspec, ref=np.max)
    librosa.display.specshow(s_db)

    savepath = os.path.join(custpath1,name+'.png')
    pylab.savefig(savepath, bbox_inches=None, pad_inches=0)
    pylab.close()
  except:
    print('File cannot open')
    return None,None
  return mfccsscaled,savepath

features_test = []
diagnoses_test = []
imgpaths_test = []

In [5]:
for row in TestData.values:
  mfcc,savepat  = feature_extractor(row)
  features_test.append(mfcc)
  imgpaths_test.append(savepat)
  diagnoses_test.append([row[1],row[2]])

In [6]:
isnone = lambda x: x is not None
label  = lambda x: 1 if x == 'positive_mild' or x =='positive_moderate' or x == 'COVID-19' else 0
cast_x = list(map(isnone,features_test))

data_x = [features_test[i] for i in range(len(features_test)) if cast_x[i] == True]
data_xx= [imgpaths_test[i] for i in range(len(imgpaths_test)) if cast_x[i] == True]
data_xp= [diagnoses_test[i]for i in range(len(diagnoses_test))if cast_x[i] == True]
data_y = [[0]]

In [7]:
DATA    = {i:{} for i in range(1)}

DATA[0]['MFCCS'] = np.array([data_x[0]])
DATA[0]['MEL']   = [data_xx[0]]
DATA[0]['EXTRA'] = np.array([data_xp[0]])
DATA[0]['LABELS'] = np.array([data_y[0]])

In [8]:
class CustomDataset(tf.keras.utils.Sequence):
  def __init__(self,imgfiles,labels,batch_size,target_size=(224,224),shuffle=False,scale=255,n_classes=1,n_channels=3):
    self.batch_size = batch_size
    self.dim        = target_size
    self.labels     = labels
    self.imgfiles   = imgfiles
    self.n_classes  = n_classes
    self.shuffle    = shuffle
    self.n_channels = n_channels
    self.scale      = scale

    self.c          = 0
    self.on_epoch_end()

  def __len__(self):
    # returns the number of batches
    return int(np.floor(len(self.imgfiles) / self.batch_size))

  def __getitem__(self, index):
    # returns one batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Generate data
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.imgfiles))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        # Store sample
        img   = cv2.imread(self.imgfiles[ID])
        img   = cv2.resize(img,self.dim,interpolation = cv2.INTER_CUBIC)
        X[i,] = img / self.scale

        # Store class
        y[i] = self.labels[ID]

        self.c +=1
    return X, y #keras.utils.to_categorical(y, num_classes=self.n_classes)


class CustomPipeline(tf.keras.utils.Sequence):
  def __init__(self,data_x,data_y,batch_size=1,shuffle=False,n_classes=1):
    self.features   = data_x
    self.labels     = data_y
    self.batch_size = 1
    self.shuffle    = shuffle
    self.n_features = self.features.shape[1]
    self.n_classes  = 1
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.features) / self.batch_size))

  def __getitem__(self,index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.features))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self,indexes):
    X = np.empty((self.batch_size, self.n_features))
    y = np.empty((self.batch_size), dtype=int)

    for i, ID in enumerate(indexes):
      X[i,] = self.features[ID]
      y[i,] = self.labels[ID]
    return X, y

class MultipleInputGenerator(tf.keras.utils.Sequence):
  """Wrapper of two generatos for the combined input model"""

  def __init__(self, X1, X2, Y, batch_size,target_size=(224,224)):
      self.genX1 = CustomPipeline(X1, Y, batch_size=batch_size,shuffle=False)
      self.genX2 = CustomDataset (X2, Y, batch_size=batch_size,shuffle=False,target_size=target_size)

  def __len__(self):
      return self.genX1.__len__()

  def __getitem__(self, index):
      X1_batch, Y_batch = self.genX1.__getitem__(index)
      X2_batch, Y_batch = self.genX2.__getitem__(index)
      X_batch = [X1_batch, X2_batch]
      return X_batch, Y_batch

class TripleInputGenerator(tf.keras.utils.Sequence):
  """Wrapper of two generatos for the combined input model"""

  def __init__(self, X1, X2, X3, Y, batch_size,target_size=(224,224)):
      self.genX1 = CustomPipeline(X1, Y, batch_size=batch_size,shuffle=False)
      self.genX2 = CustomDataset (X2, Y, batch_size=batch_size,shuffle=False,target_size=target_size)
      self.genX3 = CustomPipeline(X3, Y, batch_size=batch_size,shuffle=False)
  def __len__(self):
      return self.genX1.__len__()

  def __getitem__(self, index):
      X1_batch, Y_batch = self.genX1.__getitem__(index)
      X2_batch, Y_batch = self.genX2.__getitem__(index)
      X3_batch, Y_batch = self.genX3.__getitem__(index)

      X_batch = [X1_batch, X2_batch, X3_batch]
      return X_batch, Y_batch

In [9]:
TEST = TripleInputGenerator(DATA[0]['MFCCS'],DATA[0]['MEL'],DATA[0]['EXTRA'],DATA[0]['LABELS'],batch_size=1,target_size=(224,224))

---
# Load model
---

In [10]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/st/model_covid_resnet_auc_86% (final).h5', compile = True)

---
# Predict result
---

In [11]:
y_pred = model.predict_generator(TEST)
np.round(y_pred)

array([[0.]], dtype=float32)